In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as opt
import torch.nn.functional as F
from torchvision.io import read_image
from torchvision import transforms
from torchsummary import summary
from tqdm.notebook import tqdm
from PIL import Image
import optuna
from optuna.trial import TrialState

In [2]:
random.seed(2020)

In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [4]:
resize_image = transforms.Resize(size=(227, 227))

In [5]:
def calculate_mean(dataset):
    mean = torch.Tensor([0.,0.,0.])
    
    num_samples = len(dataset)
    for i in range(num_samples):
        image, _ = dataset[i]
        image = image.type(torch.FloatTensor) / 255
        
        for j in range(3):
            mean[j] += torch.mean(image[j, :, :])
            
    mean = mean / num_samples
    return mean

In [6]:
def subtract_mean(image, mean):
    normed_image = torch.zeros(image.size())
    image = image.type(torch.FloatTensor) / 255
    
    for j in range(3):
        normed_image[j] = image[j, :, :] - mean[j]
    return normed_image

In [7]:
class AdienceDataset(Dataset):
    def __init__(self, annot_file, img_dir, transform=None):
        self.img_lbls = pd.read_csv(annot_file, header=None)
        self.img_dir = img_dir
        self.transform = transform
        self.mean = torch.Tensor([0.,0.,0.])
    
    def __len__(self):
        return len(self.img_lbls)
    
    def __getitem__(self, idx):
        img_file = self.img_lbls.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_file)
        image = read_image(img_path)
        # image = resize_image(image)
        label = self.img_lbls.iloc[idx, 1]

        if self.transform:
            image = self.transform(image, self.mean)

        return image, label
    
    def set_mean(self, mean):
        self.mean = mean

In [8]:
# train_data = AdienceDataset("../train.csv", "../cropped_Adience/")
train_data = AdienceDataset("../train.csv", "../cropped_Adience/", transform=subtract_mean)
test_data = AdienceDataset("../test.csv", "../cropped_Adience/", transform=subtract_mean)
val_data = AdienceDataset("../val.csv", "../cropped_Adience/", transform=subtract_mean)

In [9]:
# mean = calculate_mean(train_data)
mean = torch.Tensor([0.5062, 0.3581, 0.3099])
mean

tensor([0.5062, 0.3581, 0.3099])

In [10]:
train_data.set_mean(mean)
test_data.set_mean(mean)
val_data.set_mean(mean)

In [11]:
class Net(nn.Module):
    def __init__(self, num_channels, num_classes):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(num_channels, 96, kernel_size=7, stride=4)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(256, 256, kernel_size=5, padding=2)
        self.conv4 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.norm = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)
        
        self.fc6 = nn.Linear(in_features=13824, out_features=512)
        self.fc7 = nn.Linear(in_features=512, out_features=512)
        self.fc8 = nn.Linear(in_features=512, out_features=num_classes)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        
        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
        x = self.conv4(x)
        x = self.relu(x)
        
        x = self.conv5(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
        x = torch.flatten(x, 1)
        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc7(x)
        # x_triplet = x
        x = self.relu(x)
        x = self.dropout(x)
        
        x_classification = self.fc8(x)
        
        return x_classification

In [12]:
model = Net(3, 2)
summary(model, (3, 227, 227), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 56, 56]          14,208
              ReLU-2           [-1, 96, 56, 56]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
 LocalResponseNorm-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 27, 27]         614,656
              ReLU-6          [-1, 256, 27, 27]               0
            Conv2d-7          [-1, 256, 27, 27]       1,638,656
              ReLU-8          [-1, 256, 27, 27]               0
         MaxPool2d-9          [-1, 256, 13, 13]               0
LocalResponseNorm-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 384, 13, 13]         885,120
             ReLU-12          [-1, 384, 13, 13]               0
           Conv2d-13          [-1, 384, 13, 13]       1,327,488
             ReLU-14          [-1, 384,

In [13]:
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight)

In [15]:
best_accu = 0.9175257682800293
def objective(trial):
    model = Net(3, 2)
    model.apply(init_weights)
    model = torch.jit.script(model).to(device)
    
    lr = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    mom = trial.suggest_float('momentum', 0.7, 0.99)
    wd = trial.suggest_float('weight_decay', 1e-5, 1e-1, log=True)
    optimizer = opt.SGD(model.parameters(), lr=lr, momentum=mom, weight_decay=wd)
    criterion = torch.jit.script(nn.CrossEntropyLoss())
    
    batch_size = trial.suggest_int('batch_size', 10, 100)
    num_epochs = 100
    
    print("Learning rate: "+ str(lr))
    print("Momentum: "+ str(mom))
    print("Weight decay: "+ str(wd))
    print("Batch size: "+ str(batch_size))
    print("Number of epochs: "+ str(num_epochs))
    
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
        
        # train(model, optim, train_loader)
        # training loop
        running_loss = []
        running_accu = []
        
        model.train()
        for step, (img, label) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
            img, label = img.to(device), label.to(device)

            optimizer.zero_grad()
            output = model(img)
            
            pred = torch.argmax(output, 1)
            accuracy = torch.eq(pred, label).sum() / len(img)

            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            running_accu.append(accuracy.cpu().detach().numpy())
            running_loss.append(loss.cpu().detach().numpy())
        print("Epoch: {}/{} - Loss: {:.4f} - Accuracy: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss), np.mean(running_accu)))
        
        # accuracy = test(model, val_loader)
        # validation loop
        val_loss = []
        val_accu = []

        model.eval()
        with torch.no_grad():
            for img, label in tqdm(val_loader):
                img, label = img.to(device), label.to(device)
                
                output = model(img)
                
                pred = torch.argmax(output, 1)
                accuracy = torch.eq(pred, label).sum() / len(img)
                
                loss = criterion(output, label)
                
                val_accu.append(accuracy.cpu().detach().numpy())
                val_loss.append(loss.cpu().detach().numpy())
        val_accu = np.mean(val_accu)
        val_loss = np.mean(val_loss)
        print("Val Loss: {:.4f} - Val Accuracy: {:.4f}".format(val_loss, val_accu))
        
        trial.report(val_accu, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    global best_accu
    if val_accu > best_accu:
        best_accu = val_accu
        print("Saving best model...")
        torch.save(model.state_dict(), "../incr_base_model.pt")
            
    return val_accu

In [16]:
study = optuna.create_study(direction='maximize',
                            study_name='incremented-libfaceid-study',
                            storage='sqlite:///study.db',
                            load_if_exists=True)
study.optimize(objective, n_trials=15)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

# Display the study statistics
print("\nStudy statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

[I 2023-11-09 23:19:04,624] Using an existing study with name 'incremented-libfaceid-study' instead of creating a new one.


Learning rate: 0.00011258154856445441
Momentum: 0.8363572469510143
Weight decay: 0.00018178941295912465
Batch size: 34
Number of epochs: 100


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/411 [00:00<?, ?it/s]

Epoch: 1/100 - Loss: 0.6897 - Accuracy: 0.5365


...

Training:   0%|          | 0/583 [00:00<?, ?it/s]

Epoch: 100/100 - Loss: 0.0000 - Accuracy: 1.0000


  0%|          | 0/73 [00:00<?, ?it/s]

[I 2023-11-10 10:50:23,062] Trial 19 finished with value: 0.9324133992195129 and parameters: {'learning_rate': 0.0006600637201605992, 'momentum': 0.9521566615536066, 'weight_decay': 9.201235964918673e-05, 'batch_size': 24}. Best is trial 12 with value: 0.9371428489685059.


Val Loss: 0.4747 - Val Accuracy: 0.9324

Study statistics: 
  Number of finished trials:  20
  Number of pruned trials:  5
  Number of complete trials:  15


In [17]:
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  0.9371428489685059
  Params: 
    batch_size: 10
    learning_rate: 0.001187436082338704
    momentum: 0.7800501743119681
    weight_decay: 0.0014366911365422924


In [ ]:
# after 20 trials

Best trial:
  Value:  0.9371428489685059
  Params: 
    batch_size: 10
    learning_rate: 0.001187436082338704
    momentum: 0.7800501743119681
    weight_decay: 0.0014366911365422924